In [1]:
import urllib.request
import pandas as pd
import numpy as np 
import boto3 
import sagemaker
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker import get_execution_role

!mkdir data 
!unzip ObesityDataSet.csv.zip -d data/

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
Archive:  ObesityDataSet.csv.zip
  inflating: data/ObesityDataSet.csv.zip  


In [42]:
# laoding in data to be used
df = pd.read_csv('data/ObesityDataSet.csv')
df.head()
# df.isnull().mean() * 100

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal_Weight
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal_Weight
3,Male,27.0,1.80,87.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,Frequently,Walking,Overweight_Level_I
4,Male,22.0,1.78,89.8,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation,Overweight_Level_II


In [82]:
df.isna().sum()

Gender                            0
Age                               0
Height                            0
Weight                            0
family_history_with_overweight    0
FAVC                              0
FCVC                              0
NCP                               0
CAEC                              0
SMOKE                             0
CH2O                              0
SCC                               0
FAF                               0
TUE                               0
CALC                              0
MTRANS                            0
NObeyesdad                        0
dtype: int64

In [44]:
from sklearn.model_selection import train_test_split
import os

# defining variables to be used for the model and preprocessing transformations

y1 = df["NObeyesdad"]
y2 = df["Weight"]
X_raw = df.drop(columns=["Weight", "NObeyesdad"])

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split

# Identify categorical and numeric columns
categorical_str_cols = X_raw.select_dtypes(include=['object']).columns
numeric_cols = X_raw.select_dtypes(include=['float64']).columns

# Define transformers
transformers = [
    ('categorical_str', OneHotEncoder(handle_unknown='ignore'), categorical_str_cols),
    ('numeric', StandardScaler(), numeric_cols)
]

ct = ColumnTransformer(transformers)
X_transformed = ct.fit_transform(X_raw)

# Concatenate 'y1' and 'X_transformed' to place 'y1' as the first column in the DataFrame
df_transformed = pd.concat([y1, pd.DataFrame(X_transformed)], axis=1)

# df_transformed = df_transformed.sample(frac=1).reset_index(drop=True)

df_transformed.head()


,NObeyesdad,0,1,2,3,4,5,6,7,8,...,20,21,22,23,24,25,26,27,28,29
0,Normal_Weight,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,-0.522124,-0.875589,-0.785019,0.404153,-0.013073,-1.188039,0.561997
1,Normal_Weight,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,-0.522124,-1.947599,1.088342,0.404153,1.618759,2.339750,-1.080625
2,Normal_Weight,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,-0.206889,1.054029,-0.785019,0.404153,-0.013073,1.163820,0.561997
3,Overweight_Level_I,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.423582,1.054029,1.088342,0.404153,-0.013073,1.163820,-1.080625
4,Overweight_Level_II,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,-0.364507,0.839627,-0.785019,-2.167023,-0.013073,-1.188039,-1.080625


In [84]:
df_transformed['NObeyesdad'].unique()

array([1, 2, 3, 4, 0, 5, 6])

In [56]:
# converting labels to numeric
df_transformed['NObeyesdad'] = df_transformed['NObeyesdad'].replace('Insufficient_Weight', 0)
df_transformed['NObeyesdad'] = df_transformed['NObeyesdad'].replace('Normal_Weight', 1)
df_transformed['NObeyesdad'] = df_transformed['NObeyesdad'].replace('Overweight_Level_I', 2)
df_transformed['NObeyesdad'] = df_transformed['NObeyesdad'].replace('Overweight_Level_II', 3)
df_transformed['NObeyesdad'] = df_transformed['NObeyesdad'].replace('Obesity_Type_I', 4)
df_transformed['NObeyesdad'] = df_transformed['NObeyesdad'].replace('Obesity_Type_II', 5)
df_transformed['NObeyesdad'] = df_transformed['NObeyesdad'].replace('Obesity_Type_III', 6)

df_transformed.head()

/tmp/ipykernel_7775/3606659756.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_transformed['NObeyesdad'] = df_transformed['NObeyesdad'].replace('Obesity_Type_III', 6)


,NObeyesdad,0,1,2,3,4,5,6,7,8,...,20,21,22,23,24,25,26,27,28,29
0,1,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,-0.522124,-0.875589,-0.785019,0.404153,-0.013073,-1.188039,0.561997
1,1,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,-0.522124,-1.947599,1.088342,0.404153,1.618759,2.339750,-1.080625
2,1,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,-0.206889,1.054029,-0.785019,0.404153,-0.013073,1.163820,0.561997
3,2,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.423582,1.054029,1.088342,0.404153,-0.013073,1.163820,-1.080625
4,3,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,-0.364507,0.839627,-0.785019,-2.167023,-0.013073,-1.188039,-1.080625


In [57]:
df_transformed.isna().sum()

NObeyesdad    0
0             0
1             0
2             0
3             0
4             0
5             0
6             0
7             0
8             0
9             0
10            0
11            0
12            0
13            0
14            0
15            0
16            0
17            0
18            0
19            0
20            0
21            0
22            0
23            0
24            0
25            0
26            0
27            0
28            0
29            0
dtype: int64

In [58]:
from sklearn.model_selection import train_test_split

# # Splitting data into train and test sets
# X_train, X_val, y_train, y_val = train_test_split(X_transformed, y1, test_size=0.2, random_state=0, stratify=y1)

# # Combine X and y for train and val sets
# train_data = pd.concat([pd.DataFrame(X_train), pd.DataFrame(y_train)], axis=1)
# val_data = pd.concat([pd.DataFrame(X_val), pd.DataFrame(y_val)], axis=1)

train_size = int(0.8 * len(df_transformed))
val_size = len(df_transformed) - train_size

# Splitting df_transformed into train and validation sets using indexing
train_data = df_transformed[:train_size]
val_data = df_transformed[train_size:]

# Print the shapes of train, val, and test sets
print("Train data shape:", train_data.shape)
print("Val data shape:", val_data.shape)
# print("Test data shape:", X_test.shape)

Train data shape: (1688, 31)
Val data shape: (423, 31)


In [61]:
# defining bucket to be used in AWS as well as some other session details 
bucket_name = 'sagemaker-bucket-jud2'
train_data.to_csv('health_train_data.csv', header=False, index=False)
key = 'data/health_train/health_train_data'
url = 's3://{}/{}'.format(bucket_name, key)

boto3.Session().resource('s3').Bucket(bucket_name).Object(key).upload_file('health_train_data.csv')

train_data.to_csv('health_val_data.csv', header=False, index=False)
key = 'data/health_val/health_val_data'
url = 's3://{}/{}'.format(bucket_name, key)

boto3.Session().resource('s3').Bucket(bucket_name).Object(key).upload_file('health_val_data.csv')

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [62]:
train_data['NObeyesdad'].unique()

array([1, 2, 3, 4, 0, 5, 6])

In [63]:
import sagemaker
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker import get_execution_role
# using predefined xgboost model in AWS for this example, defined below with certain hyperparameters that could 
# be tuned in the future should this project be continued

key = 'model/xgb_health_model'
s3_output_location = url = 's3://{}/{}'.format(bucket_name, key)

xgb_model = sagemaker.estimator.Estimator(
    get_image_uri(boto3.Session().region_name, 'xgboost'),
    get_execution_role(),
    train_instance_count =1, 
    train_instance_type= 'ml.m4.xlarge', 
    train_volume_size=5, 
    output_path = s3_output_location, 
    sagemaker_session = sagemaker.Session()
)

# setting hyperparameters for this approach, can see various choices 
# in the aws documentations for the various models 

xgb_model.set_hyperparameters(
    max_depth=5,
    eta=.2,
    gamma=4, 
    min_child_weight=6, 
    silent=0, 
    objective='multi:softmax',   # output activation
    num_class=7, # update depending on data
    num_round=10  # essentially epochs
)

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [64]:
# referencing the directories where the data is saved in the bucket
train_data = 's3://{}/{}'.format(bucket_name, 'data/health_train')
val_data = 's3://{}/{}'.format(bucket_name, 'data/health_val')

# defining channels to use for training and validation
train_channel = sagemaker.session.s3_input(train_data, content_type = 'text/csv')
val_channel = sagemaker.session.s3_input(val_data, content_type = 'text/csv')

data_channels = {'train': train_channel, 'validation': val_channel}

# fitting model
xgb_model.fit(inputs = data_channels)

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker:Creating training-job with name: xgboost-2024-05-02-18-19-45-371


2024-05-02 18:19:45 Starting - Starting the training job...
2024-05-02 18:20:01 Starting - Preparing the instances for training...
2024-05-02 18:20:41 Downloading - Downloading input data...
2024-05-02 18:21:15 Downloading - Downloading the training image......
2024-05-02 18:22:06 Training - Training image download completed. Training in progress..Arguments: train
[2024-05-02:18:22:18:INFO] Running standalone xgboost training.
[2024-05-02:18:22:18:INFO] File size need to be processed in the node: 0.75mb. Available memory size in the node: 8474.39mb
[2024-05-02:18:22:18:INFO] Determined delimiter of CSV input is ','
[18:22:18] S3DistributionType set as FullyReplicated
[18:22:18] 1688x30 matrix with 50640 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2024-05-02:18:22:18:INFO] Determined delimiter of CSV input is ','
[18:22:18] S3DistributionType set as FullyReplicated
[18:22:18] 1688x30 matrix with 50640 entries loaded from /opt/ml/input/data/validat

In [65]:
# deploying model to an AWS endpoint that could be accessed within a Lambda Function, and 
# utilized within an API created in AWS API Gateway! 
xgb_predictor = xgb_model.deploy(initial_instance_count=1, 
                                instance_type = 'ml.m4.xlarge')

INFO:sagemaker:Creating model with name: xgboost-2024-05-02-18-25-31-393
INFO:sagemaker:Creating endpoint-config with name xgboost-2024-05-02-18-25-31-393
INFO:sagemaker:Creating endpoint with name xgboost-2024-05-02-18-25-31-393


------!